# **Práctica 4.1: Redes neuronales (Regresión)**

<hr>

## **1. Introducción**
En prácticas anteriores aprendimos a resolver problemas de clasificación y regresión utilizando métodos o modelos de aprendizaje automático clásico. En esta práctica 4 veremos cómo resolver ese tipo de problemas utilizando **Redes Neuronales**, los métodos más utilizados en la actualidad. 

Estas redes son la base del *Deep Learning* y tienen la capacidad de ser muy versátiles al estar compuestas por múltiples neuronas organizadas en capas. A estas neuronas también se les denomina *perceptrones*, es por eso que a las redes neuronales también se les conoce como **Perceptrones Multicapa**.

Al igual que sucedía con el resto de modelos, dentro de `scikit-learn` tenemos un par de clases que nos facilitan el uso de estas redes:

* **MLPRegressor:** Perceptrón multicapa pensado para resolver problemas de regresión.
* **MLPClassifier:** Perceptrón multicapa pensado para resolver problemas de clasificación.

En este tipo de modelos no vamos a utilizar estas clases predefinidas, ya que nos interesa ver cómo es su arquitectura y funcionamiento interno. Por este motivo utilizaremos una nueva librería, [`tensorflow`](https://www.tensorflow.org/), la cual es una de las más utilizadas en Python para llevar a cabo tareas de deep learning junto con [`pytorch`](https://pytorch.org/).

### **Objetivos**
En esta práctica aprenderás a:
* Crear y entrenar Redes Neuronales.
* Optimizar sus hiperparámetros.
* Añadir funciones de activación no lineales.

Comenzaremos instalando la librería en nuestro environment.


In [ ]:
! pip install tensorflow

Una de las ventajas de las redes neuronales es que se pueden entrenar en CPU o GPU. 

Más adelante veremos como entrenar una de estas redes en las GPUs de los ordenadores de prácticas buscando acelerar el proceso de entrenamiento. En tu ordenador personal, es probable solo puedas ejecutar en CPU.

A continuación cargamos una vez más nuestros datos:

In [ ]:
import pandas as pd

seed = 2533
data = pd.read_pickle("https://raw.githubusercontent.com/AIC-Uniovi/Sistemas-Inteligentes/refs/heads/main/datasets/f1_23_monaco.pkl")

<hr>

## **2. Problemas de regresión**

Vamos a intentar resolver el mismo problema de la práctica anterior, es decir: 

<div class="alert alert-block alert-success">
    <b>Crear un modelo que, dado el tiempo en el primer sector <code>Sector1Time</code> sea capaz de predecir el tiempo total de la vuelta <code>LapTime</code>.</b>
</div>

Lo primero que haremos será crear los datasets necesarios para entrenar un modelo.

### **2.1. Preprocesado de datos**


<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa las X e Y del dataframe <code>data_sector2lap</code> , divide en entrenamiento y test (80/20) fijando la semilla y finalmente <b>estandariza</b> las X.
    <hr>
    Cuando la X solo tiene una columna, es necesario utilizar dobles corchetes (<code>data[["nombrecolumna"]]</code> y no <code>data["nombrecolumna"]</code>) para que funcione correctamente el <code>StandardScaler()</code>.
</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data_sector2lap = data[["LapTime", "Sector1Time"]].copy()
data_sector2lap["LapTime"] = data_sector2lap["LapTime"].dt.total_seconds()
data_sector2lap["Sector1Time"] = data_sector2lap["Sector1Time"].dt.total_seconds()

# Tu código aquí

### **2.2. Aprendizaje automático**

Con los datos listos, entrenaremos y evaluaremos de nuevo los modelos de aprendizaje automático ya conocidos para poder compararlos con nuestro nuevo sistema.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entrena y evalúa los modelos restantes (<i>K-Nearest Neighbors</i>, <i>Árboles de decisión</i> y <i>SVR</i>) utilizando la siguiente función.
</div>

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

def evaluate_model(Y_test, preds_test, model_name):
    metrics = {
        "Métrica": ["MAE", "MSE", "R²"],
        "TEST": [mean_absolute_error(Y_test, preds_test), 
                 mean_squared_error(Y_test, preds_test), 
                 r2_score(Y_test, preds_test)]
    }
    df = pd.DataFrame(metrics)
    print(f"Resultados para {model_name}:")
    print(df.to_string(index=False))
    print()

# Baseline media
baseline_media = DummyRegressor(strategy="mean")
baseline_media.fit(X_train, Y_train)
preds_test = baseline_media.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline")

# Regresión Lineal
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
preds_test = model_linear.predict(X_test)
evaluate_model( Y_test, preds_test, "Lineal")

# Tu código aquí

Los resultados han de ser algo así:

<center>

| Modelo                 | MAE Test  | MSE Test  | R² Test  |
|------------------------|-----------|-----------|----------|
| *Baseline*             | 5.882     | 52.489    | -0.008   |
| *Lineal*               | 1.056     |  2.298    |  0.956   |
| *KNN*                  | 0.820     |  1.801    |  0.965   |
| *Árboles de Decisión*  | 1.110     |  3.575    |  0.931   |
| *SVR*                  | 0.735     |  1.738    |  0.967   |

</center>

### **2.3. Red neuronal**

Una vez tenemos los conjuntos y los resultados de los modelos tradicionales ya podemos crear nuestra primera red neuronal desde cero.

Los pasos para crear y entrenar una red neuronal son los siguientes:

1) Crear la arquitectura del modelo.
2) Detallar el optimizador, la función de pérdida y compilar.
3) Entrenar y evaluar.

#### **2.3.1. Crear arquitectura**

Lo primero que tenemos que definir es su arquitectura, es decir, el número de capas completamente conectadas que la forman. 

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Como ya sabes, el tamaño de la capa de entrada y la capa de salida viene determinado por el problema a resolver. En este caso tenemos 1 entrada y una salida.
</div>

La red más sencilla en este caso será la siguiente:

In [ ]:
import os, random
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Fijar las semillas de las librerías para que los resultados se repitan.
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Definir las capas del modelo
model = Sequential()
model.add(Input(shape =(1,)))
model.add(Dense(1, name="output_layer"))

El código anterior simplemente crea un modelo `Sequential()`, es decir, un modelo donde las capas se añaden una después de otra. 

A continuación vamos introduciendo las siguientes capas en el mismo:

* `Input()`: Capa de entrada. No es una capa como tal, simplemente sirve para que el modelo conozca el tamaño de las entradas.
* `Dense()`: Capa totalmente conectada ( $y=Wx+b$ ). Le indicamos el tamaño como parámetro obligatorio, el `name` es opcional.

Para ver la arquitectura final de nuestro modelo podemos ejecutar el método `summary()` que además nos dá información sobre los parámetros.

In [ ]:
model.summary()

Cabe destacar que, como mencionamos previamente, al no ser una capa como tal, `Input()` ni siquiera aparece. 

Otro dato relevante es el número de parámetros o pesos del modelo, es decir, cuantos $W$ y $b$ tiene que aprender durante el entrenamiento para transformar correctamente la entrada en la salida. 

En este caso son solo 2, un $w$ y un $b$.

#### **2.3.2. Compilar el modelo**


La compilación de un modelo en Keras es un paso esencial antes de entrenarlo. Sirve para configurar el proceso de aprendizaje, especificando cómo se optimizará el modelo y cómo se evaluará su rendimiento. En esencia, define los siguientes elementos clave:

*   **Optimizador (Optimizer):** Define el algoritmo que se utilizará para ajustar los pesos del modelo durante el entrenamiento, buscando minimizar la función de pérdida. Ejemplos comunes son `Adam`, `SGD`, `RMSprop`, entre otros. Cada optimizador tiene sus propios hiperparámetros (como el learning rate) que pueden ajustarse.

*   **Función de Pérdida (Loss Function):**  Mide la diferencia entre las predicciones del modelo y los valores reales (etiquetas). El objetivo del entrenamiento es **minimizar esta pérdida**. La elección de la función de pérdida depende del tipo de problema (clasificación, regresión, etc.). Ejemplos: `categorical_crossentropy` (para clasificación multiclase), `binary_crossentropy` (para clasificación binaria o multietiqueta), `mean_squared_error` o `mean_absolute_error` (para regresión).

A continuación compilamos el modelo indicando el optimizador `Adam` con un `learning_rate=0.05` y una función de pérdida de regresión (`mean_absolute_error`).

In [ ]:
from tensorflow.keras.optimizers import Adam

learning_rate = 0.001
optim = Adam(learning_rate=learning_rate)
model.compile(loss='mean_absolute_error', optimizer=optim )

#### **2.3.3. Entrenar y evaluar**


El siguiente paso es el entrenamiento y evaluación del modelo. En `keras` el entrenamiento se realiza a través del método [`.fit()`](https://keras.io/api/models/model_training_apis/), al igual que sucedía en `scikit-learn`.

Este método también recibe los datos de entrenamiento ($X$ e $Y$) y ajusta los pesos del modelo (las $W$ y los $b$) iterativamente para minimizar la función de pérdida (loss function).

En detalle, este método realiza los siguientes pasos:

1) **Creación de batches:** Divide todo el conjunto de entrenamiento en batches (bloques de ejemplos) y obtiene la predicción del modelo para cada batch. Las redes neuronales están pensadas para trabajar con conjuntos de datos muy grandes y muchas veces no es posible entrenar con todo el conjunto a la vez como hacíamos en los métodos de aprendizaje automático anteriores.
2) **Calcula la loss:** Compara las predicciones del modelo $\hat{Y}$ con las etiquetas reales $Y$ y calcula la pérdida indicada a la hora de compilar, $MAE$ en este caso.
3) **Calcula los gradientes:** Utiliza el algoritmo de backpropagation para calcular los gradientes de la función de pérdida con respecto a los pesos del modelo.  Los gradientes indican la dirección y magnitud del cambio necesario en los pesos para reducir la loss.

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:900px">
        <img src="https://i.imgur.com/1tscXrJ.png">   
    </div>
</center>

4) **Actualiza los pesos:**  Ajusta los pesos del modelo utilizando el optimizador basado en los gradientes calculados.  El optimizador determina cómo se actualizan los pesos para minimizar la pérdida de manera eficiente.
5) **Repite el proceso:** Repite los pasos anteriores para cada uno de los batchs de entrenamiento. Cuando se acaban los batches, se puede volver a repetir todo el proceso durante un número determinado de épocas (epochs).

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Una época (epoch) representa una pasada completa por todo el conjunto de datos de entrenamiento.
</div>

A continuación entrenamos el modelo con los datos de entrenamiento:

In [ ]:
# Entrenar el modelo. el parámetro verbose del método train permite configurar la cantidad de información que se muestra por consola durante el entrenamiento
history = model.fit(X_train, Y_train, validation_split=0.2, batch_size=64, epochs=200, verbose=2)

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Cada vez que ejecutes el bloque anterior <strong>se continuará</strong> con el entrenamiento en el punto que había finalizado. Para entrenar el modelo desde cero tendrías que volver a crearlo y compilarlo.
</div>

Como puedes ver, hay un argumento en el método `fit` del que no hemos hablado: `validation_split`. 

Este argumento reserva una fracción de los datos de entrenamiento para usarla como conjunto de validación; es decir, separa un $20\%$ de los ejemplos del conjunto de entrenamiento (en este caso, dado que `validation_split=0.2`).

Por tanto ahora nuestro conjunto de datos está dividido en tres bloques distintos: Train, Validation y Test. Este enfoque también se conoce como **metavalidación**.

##### **Metavalidación**

Hasta ahora, para validar nuestro modelo habíamos empleado la **validación simple** o hold-out, es decir, habíamos dividido en: Train y Test.

Este enfoque es muy válido para casos donde los modelos que queremos evaluar no tienen **hiperparámetros**, como la regresión lineal.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> El conjunto de validación sirve para ajustar los hiperparámetros del modelo. <hr>
    Recuerda que un hiperparámetro es todo aquel valor que nosotros podemos indicar a la hora de crear un modelo, por ejemplo la K del KNN.
</div>

Las redes neuronales poseen múltiples hiperparámetros que ajustar como `learning rate`, `batch size`, `epochs`, `número de capas`, `número de neuronas`, etc. Por eso es necesario este conjunto de validación.

Por tanto ahora, el proceso de entrenamiento cambia ligeramente: A partir de los datos de Train, el modelo calcula la loss y actualiza los pesos según corresponda, a continuación *calcula la pérdida en el conjunto de validación* **pero no actualiza los pesos con estos datos**.

Esto permite:

* **Monitorizar el sobreajuste (overfitting):**  Si el rendimiento en el conjunto de entrenamiento mejora continuamente, pero el rendimiento en el conjunto de validación se estanca o empeora, esto indica que el modelo está empezando a sobreajustarse a los datos de entrenamiento y perdiendo su capacidad de generalizar a datos nuevos.
* **Ajustar hiperparámetros:**  Ajustaremos los valores de los hiperparámetros buscando seleccionar la configuración que produce el mejor rendimiento en los datos de validación.

<hr>

##### **¿Por qué no utilizamos directamente Test para ajustar los hiperparámetros?**

Imagina que quieres preparar el bizcocho perfecto para llevarlo a un concurso de repostería.

* **Conjunto de entrenamiento (train)** Son todas las pruebas que haces en casa con distintas recetas, temperaturas y tiempos. Aquí pruebas muchas combinaciones para aprender cómo afecta cada cambio.

* **Conjunto de validación (val)** Cada vez que haces un bizcocho en casa, dejas que lo prueben tus amigos o familia. Ellos te dicen si está muy seco, muy dulce, si la textura es buena... Con ese feedback ajustas la receta: ¿más azúcar? ¿menos tiempo de horno? Ese es el proceso de ajustar hiperparámetros.

* **Conjunto de prueba (test)** Es el jurado del concurso. Nunca les diste a probar ninguno de tus bizcochos. Ellos evaluarán si, más allá de tus ajustes, tu receta funciona de verdad.

**¿Dónde estaría la trampa?**

Si, antes del concurso, tú haces que el jurado pruebe varios bizcochos y te diga qué cambiar, cuando vayas al concurso, ya sabrás lo que les gusta. 
No estarás probando si tu receta es buena en general, sino que habrás hecho un bizcocho a medida para ellos.

<hr>


En resumen: 
* **Train:** Conjunto de datos empleado para entrenar el modelo, ajustando sus pesos con el fin de minimizar la función de pérdida. 
* **Validation:** Conjunto de datos utilizado para monitorizar el sobreajuste y optimizar los hiperparámetros del modelo, **pero no los pesos**. 
* **Test:** Conjunto de datos empleado para evaluar el rendimiento final del modelo entrenado con los hiperparámetros óptimos y datos no utilizados previamente (ni para entrenar ni para ajustar hiperparámetros).

El método `fit()` retorna un objeto que almacena todo el historial del entrenamiento del modelo a lo largo de las epochs. En este caso almacenamos el resultado en `history`.

Vamos a crear una función para visualizar la evolución de las losses de entrenamiento y validación:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_loss_history(history):
    # Extraer datos del historial
    loss = history.history['loss']
    val_loss = history.history.get('val_loss', []) # Usamos lista vacía por defecto
    epochs = range(1, len(loss) + 1)
    # Construir el DataFrame
    data = pd.DataFrame({ 'Epoch': epochs, 'Loss': loss, 'Type': 'Train' })
    # Si existe validación, concatenamos esos datos
    if val_loss:
        val_df = pd.DataFrame({ 'Epoch': epochs, 'Loss': val_loss, 'Type': 'Validation' })
        data = pd.concat([data, val_df], ignore_index=True)
    # Crear el gráfico
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=data, x="Epoch", y="Loss", hue="Type")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Evolución de la Pérdida durante el Entrenamiento")
    plt.grid(True, linestyle='--', alpha=0.7)
    # Solo mostrar leyenda si hay más de un tipo de dato
    if val_loss: plt.legend(title="Conjunto")
    else: plt.legend().remove() # Evita leyendas vacías o innecesarias
    plt.show()

# Llamada a la función (después de entrenar el modelo)
plot_loss_history(history)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Por simplicidad vamos juntar la creación y complilación de la red dentro de una función. Completa el siguiente código y vuelve a entrenar red desde cero. 
</div>

In [ ]:
def red_neuronal_uno(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí

    return model

# Creamos la red desde cero
model_1 = red_neuronal_uno(learning_rate = 0.001)

# Entrenamos
# Tu código aquí

# Visualizar entrenamiento
# Tu código aquí

##### **Ajuste de hiperparámetros**

Una vez tenemos todo junto en un único bloque de código, podemos pasar a la parte de ajuste de hiperparámetros. 

Hay muchas cosas que podemos ajustar ( `batch size`, `epochs`, `learning rate`, ...) pero vamos a centrarnos en el `learning rate`.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Anota los resultados de la última epoch del modelo anterior en la siguiente tabla y realiza los experimentos necesarios para completar el resto de filas.
</div>


<center>

| Modelo                    | MAE Train  | MAE Val |
|---------------------------|------------|---------|
| *Red Neuronal (lr=0.001)* |            |         |
| *Red Neuronal (lr=0.05)*  |            |         |
| *Red Neuronal (lr=0.1)*   |            |         |

</center>



In [ ]:
# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entrena el modelo final con el mejor <code>learning_rate</code> y evalúa en test utilizando el método <code>.predict()</code> y la función <code>evaluate_model()</code> creada previamente.
    <hr>
    Añade los resultados a la tabla.
</div>

<center>

| Modelo                 | MAE Test  | MSE Test  | R² Test  |
|------------------------|-----------|-----------|----------|
| *Baseline*             | 5.882     | 52.489    | -0.008   |
| *Lineal*               | 1.056     |  2.298    |  0.956   |
| *KNN*                  | 0.820     |  1.801    |  0.965   |
| *Árboles de Decisión*  | 1.110     |  3.575    |  0.931   |
| *SVR*                  | 0.735     |  1.738    |  0.967   |
| *Red Neuronal Lineal*  |           |           |          |

</center>


In [ ]:
# Tu código aquí

##### **Añadiendo no linealidad**

Realizaremos algunas modificaciones y crearemos un modelo distinto, algo más complejo, para comprobar si podemos mejorar el rendimiento.

Como recordarás, no todos los problemas tienen una solución lineal. Una red neuronal básica como la nuestra se limita a realizar combinaciones lineales de números (sumas y multiplicaciones), es decir, transforma la entrada en la salida mediante la fórmula $y=Wx + b$.

Para añadir no linealidad, necesitamos introducir **funciones de activación no lineales**. Estas funciones se aplican a la salida de cada capa y permiten que la red aprenda relaciones más complejas entre las entradas y las salidas. Algunas funciones de activación comunes incluyen `ReLU`, `sigmoid` y `tanh`. Al agregar estas funciones, la red puede aproximar funciones no lineales y resolver problemas más desafiantes.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Si queremos que el modelo aprenda relaciones <b>no lineales</b>, necesitamos introducir estas funciones de activación no lineales <b>en capas ocultas, no solo en la de salida</b>.
</div>

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:900px">
        <img src="https://i.imgur.com/e7kd5fs.png">   
    </div>
</center>

Nuestro actual modelo solo tiene una capa, la de salida. Si pusiesemos cualquiera de estas funciones en esta capa, **además de no conseguir no linealidad**, estaríamos restringiendo significativamente el rango de valores que el modelo puede predecir. 

Por ejemplo, si solo añadimos una función `sigmoide` en la capa final, tendremos un modelo lineal cuya salida **siempre** se limitará al rango entre 0 y 1. Esto no es lo ideal para nuestro problema de regresión. 

Para conseguir no linealidad y evitar este inconveniente permitiendo que el modelo prediga en un rango de valores entre $(-\infty , \infty)$, añadiremos una capa oculta (o intermedia) y aplicaremos la activación en esta capa. De esta manera, la capa final puede generar salidas sin las restricciones impuestas por la función de activación, al tiempo que conseguimos la no linealidad.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea, dentro de la función proporcionada, una nueva red neuronal, pero este vez con dos capas siendo la oculta de <b>tamaño 10</b> y con una función de activación <code>tanh</code> para añadir no linealidad. 
    <br>
    Consulta la documentación de la capa <a href="https://keras.io/api/layers/core_layers/dense/"><code>Dense()</code></a>.
    <hr>
    Busca el mejor <code>learning_rate</code> y evalúa en test con la mejor versión. Rellena ambas tablas.
</div>


<center>

| Modelo                    | MAE Train  | MAE Val |
|---------------------------|------------|---------|
| *Red Neuronal (lr=0.001)* |            |         |
| *Red Neuronal (lr=0.05)*  |            |         |
| *Red Neuronal (lr=0.1)*   |            |         |


<center>
<br> 
<center>

| Modelo                   | MAE Test  | MSE Test  | R² Test  |
|--------------------------|-----------|-----------|----------|
| *Baseline*               | 5.882     | 52.489    | -0.008   |
| *Lineal*                 | 1.056     |  2.298    |  0.956   |
| *KNN*                    | 0.820     |  1.801    |  0.965   |
| *Árboles de Decisión*    | 1.110     |  3.575    |  0.931   |
| *SVR*                    | 0.735     |  1.738    |  0.967   |
| *Red Neuronal Lineal*    |           |           |          |
| *Red Neuronal No Lineal* |           |           |          |

</center>


In [ ]:
def red_neuronal_dos(learning_rate):
    # Creamos y compilamos el modelo
    
    # Tu código aquí

    return model

# Creamos la red desde cero
model_2 = red_neuronal_dos(learning_rate = 0.001)

# Tu código aquí

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> ¿Con qué modelo te quedarías?.
</div>

Tu respuesta aquí

### **2.4. Múltiples entradas**

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea un par de redes capaces de predecir <i>LapTime</i> a parir de <i>SpeedI1</i>, <i>SpeedI2</i>, <i>SpeedFL</i>, <i>SpeedST</i> y <i>TyreLife</i>. Ajusta los hiperparámetros de cada modelo.
</div>

<center>

| Modelo                   | MAE Test  | MSE Test  | R² Test  |
|--------------------------|-----------|-----------|----------|
| *Baseline*               | 5.858     | 54.094    | -0.008   |
| *Lineal*                 | 0.932     | 1.678     | 0.969    |
| *KNN*                    | 0.785     | 1.743     | 0.968    |
| *Árboles de Decisión*    | 0.899     | 1.841     | 0.966    |
| *SVR*                    | 0.938     | 2.724     | 0.949    |
| *Red Neuronal 1*         |           |           |          |
| *Red Neuronal 2*         |           |           |          |

</center>

In [ ]:
data_lap_time = data[["LapTime", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "TyreLife"]].copy()
data_lap_time = data_lap_time.dropna()
data_lap_time["LapTime"] = data_lap_time["LapTime"].dt.total_seconds()

X = data_lap_time[["SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "TyreLife"]]
Y = data_lap_time["LapTime"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=seed, test_size=.2)

standardizer = StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

# Baseline media
baseline_media = DummyRegressor(strategy="mean")
baseline_media.fit(X_train, Y_train)
preds_test = baseline_media.predict(X_test)
evaluate_model(Y_test, preds_test, "Baseline")

# Regresión Lineal
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
preds_test = model_linear.predict(X_test)
evaluate_model( Y_test, preds_test, "Lineal")

# KNN
model_knn = KNeighborsRegressor()
model_knn.fit(X_train, Y_train)
preds_test = model_knn.predict(X_test)
evaluate_model(Y_test, preds_test, "KNN")

# Árboles de Decisión
model_tree = DecisionTreeRegressor()
model_tree.fit(X_train, Y_train)
preds_test = model_tree.predict(X_test)
evaluate_model(Y_test, preds_test, "Árboles de Decisión")

# SVR
model_svr = SVR()
model_svr.fit(X_train, Y_train)
preds_test = model_svr.predict(X_test)
evaluate_model(Y_test, preds_test, "SVR")

In [ ]:
# Tu código aquí